### SPARQL Type Checking
PyShEx 作者リタイアした、JoseがRustで作ってて今後pytonで更新されるらしい  
rdflib SPARQLをストアして木構造作る

In [1]:
# load question_uniprot.json
import json
with open('question_uniprot.json') as f:
    data = json.load(f)

# load sparql.json
with open('sparql.json', 'r') as f:
    sparqls = json.load(f)

In [2]:
last_id = 0
for i in range(len(sparqls)):
    if last_id < sparqls[i]['id']:
        last_id = sparqls[i]['id']

id_index = last_id + 1
for i in range(len(data)):
    if "-1-" not in data[i]['id']:
        continue
    sparqls += [{
        "id": id_index,
        "sparql": data[i]['sparql'],
        "label": True
    }]
    id_index += 1


In [3]:
sparqls

[{'id': 1,
  'sparql': 'PREFIX core: <http://purl.uniprot.org/core/>\nPREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nSELECT DISTINCT ?uniprot_pdb \nWHERE {\n    VALUES ?UniProt__class { core:Protein chembl:UniprotRef }\n    ?UniProt a ?UniProt__class ;\n        rdfs:seeAlso ?uniprot_pdb.\n    ?uniprot_pdb a core:Structure_Resource .\n\n}\nLIMIT 100\n',
  'label': True},
 {'id': 2,
  'sparql': 'PREFIX core: <http://purl.uniprot.org/core/>\n\nSELECT DISTINCT ?UniProt \nWHERE {\n    ?UniProt a core:Protein ;\n        ?x "2014-09-03"^^xsd:date\n}\nLIMIT 100\n',
  'label': True},
 {'id': 3,
  'sparql': '\nPREFIX core: <http://purl.uniprot.org/core/>\nPREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n\nSELECT DISTINCT ?uniprot_seeAlso\nWHERE {\n    VALUES ?UniProt__class { core:Protein chembl:UniprotRef }\n    ?UniProt a ?UniProt__class ;\n        rdfs:seeAlso ?uniprot_seeAl

In [4]:
# save as sparqls.json
with open('sparqs.json', 'w') as f:
    json.dump(sparqls, f)

In [5]:
from pyshex import ShExEvaluator
import re
import json
from rdflib import URIRef, BNode, Literal
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.plugins.sparql import prepareQuery
from rdflib.term import Variable

In [6]:
with open('uniprot_rdf-config.shex', 'r') as f:
    ShEx_uniprot = f.read()

In [7]:
ShEx_uniprot

'PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX dct: <http://purl.org/dc/terms/>\nPREFIX obo: <http://purl.obolibrary.org/obo/>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\nPREFIX foaf: <http://xmlns.com/foaf/0.1/>\nPREFIX faldo: <http://biohackathon.org/resource/faldo#>\nPREFIX up: <http://purl.uniprot.org/uniprot/>\nPREFIX core: <http://purl.uniprot.org/core/>\nPREFIX taxonomy: <http://purl.uniprot.org/taxonomy/>\nPREFIX enzyme: <http://purl.uniprot.org/enzyme/>\nPREFIX citations: <http://purl.uniprot.org/citations/>\nPREFIX pubmed: <https://pubmed.ncbi.nlm.nih.gov/>\nPREFIX keywords: <http://purl.uniprot.org/keywords/>\nPREFIX isoforms: <http://purl.uniprot.org/isoforms/>\nPREFIX rhea: <http://rdf.rhea-db.org/>\nPREFIX dbsnp: <http://purl.uniprot.org/dbsnp/>\nPREFIX locations: <http://purl.uniprot.org/locatio

In [24]:
import sys
sys.stderr = open('/dev/null', 'w')
# トリプル抽出用の関数
def extract_triples_from_shex(shex_text):
    triples = []
    # シェイプごとに分割
    shapes = re.split(r'<([^>]+)> {', shex_text)
    
    for i in range(1, len(shapes), 2):
        shape_name = f"<{shapes[i]}>"
        shape_body = shapes[i + 1].strip().rstrip('}')
        
        # 述語と目的語のペアを抽出
        pairs = re.findall(r'<([^>]+)> (\[[^]]+\]|BNODE|IRI|@<[^>]+>|<[^>]+>)', shape_body)
        
        for pred, obj in pairs:
            # BNODE, IRI, リテラルを適切に処理
            if obj == 'BNODE':
                obj = BNode()
            elif obj == 'IRI':
                obj = URIRef('IRI')
            elif obj.startswith('[') and obj.endswith(']'):
                obj = URIRef(obj[2:-2])
            elif obj.startswith('@<') and obj.endswith('>'):
                obj = URIRef(obj[1:])
            else:
                obj = URIRef(obj[1:-1])
            
            triples.append((URIRef(shape_name), URIRef(pred), obj))
    
    return triples

shex = ShExEvaluator(schema=ShEx_uniprot)
# トリプルを抽出
shex_triples = extract_triples_from_shex(shex.schema)

# if shex_triples has "<http:....>" uri then make "http:...."
def uri_to_str(uri):
    uri = str(uri)
    if uri.startswith('<http'):
        return uri[1:-1]
    return uri

# use uri_to_str and update shex_triples
shex_triples = [(uri_to_str(subj), uri_to_str(pred), uri_to_str(obj)) for subj, pred, obj in shex_triples]

In [9]:
shex_triples

[('<UniProtShape>',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://purl.uniprot.org/core/Protein'),
 ('<UniProtShape>',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://rdf.ebi.ac.uk/terms/chembl#UniprotRef'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/recommendedName',
  'Nb5b6aa629fb748d4afe5ddf65d2b6824'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/encodedBy',
  'N7e5c165d45404ec6aaafffa95130bc09'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/created',
  'http://www.w3.org/2001/XMLSchema#date'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/modified',
  'http://www.w3.org/2001/XMLSchema#date'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/annotation',
  '<FunctionAnnotationShape>'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/annotation',
  '<BindingSiteAnnotationShape>'),
 ('<UniProtShape>',
  'http://purl.uniprot.org/core/annotation',
  '<CatalyticActivityAnnotationShape>'),
 ('<UniProtShape>',
  'http://

In [10]:
# load sparqls
# with open('sparql.json') as f:
#     sparqls = json.load(f)

In [11]:
# クエリ内の変数を抽出
def extract_variables_from_query(query):
    # Projection variables
    projected_vars = {str(var) for var in query.algebra['PV']}
    
    # Internal variables in query body
    internal_vars = {str(var) for var in query.algebra['_vars']}
    
    # Combine both sets
    all_vars = projected_vars.union(internal_vars)
    
    return all_vars, projected_vars, internal_vars

### トリプルの抽出

In [12]:
# トリプルを抽出する関数
def extract_triples(node):
    triples = []
    if isinstance(node, dict):
        for key, value in node.items():
            if key == 'triples':
                triples.extend(value)
            else:
                triples.extend(extract_triples(value))
    elif isinstance(node, list):
        for item in node:
            triples.extend(extract_triples(item))
    return triples

In [13]:
def candiate_update(candidates, _candidates, verbose):
    if verbose: print("candidates", candidates, "\n_candidates", _candidates)
    if candidates != [-1]:
        # get & of candidates and _candidates
        candidates = list(set(candidates) & set(_candidates))
    else:
        candidates = _candidates
    
    #print("retuern", candidates)
    return candidates

def find_matching_classes(var, sparql_triples, shex_triples, verbose=False):
    """
    Given a SPARQL variable, find matching classes in the ShEx triples.
    """
    if verbose: print("var", var)
    rdflb_var = Variable(var)
    candidate_classes = [-1]

    for sparql_triple in sparql_triples:
        if rdflb_var not in sparql_triple:
            continue

        s, p, o = sparql_triple
        #print("s", s, "p", p, "o", o)

        # Check subject (s) matches
        if s == rdflb_var:
            if isinstance(p, URIRef):
                if verbose: print("Pos 1")
                candidate_classes = candiate_update(candidate_classes, [shex_triple[0] for shex_triple in shex_triples if URIRef(shex_triple[1]) == p], verbose)
            if isinstance(o, URIRef):
                if verbose: print("Pos 2")
                candidate_classes = candiate_update(candidate_classes, [shex_triple[0] for shex_triple in shex_triples if URIRef(shex_triple[2]) == o], verbose)
            if isinstance(o, Literal):
                if verbose: print("Pos 3")
                candidate_classes = candiate_update(candidate_classes, [shex_triple[0] for shex_triple in shex_triples if URIRef(shex_triple[2]) == o.datatype], verbose)
            if isinstance(o, BNode):
                candidate_classes = candiate_update(candidate_classes, [shex_triple[0] for shex_triple in shex_triples if BNode == type(o)], verbose)
        
        # Check object (o) matches
        if o == rdflb_var:
            if isinstance(s, BNode):
                continue
            if isinstance(p, URIRef):
                if verbose: print("Pos 4")
                candidate_classes = candiate_update(candidate_classes, [shex_triple[2] for shex_triple in shex_triples if URIRef(shex_triple[1]) == p], verbose)
            if isinstance(s, URIRef):
                if verbose: print("Pos 5")
                candidate_classes = candiate_update(candidate_classes, [shex_triple[2] for shex_triple in shex_triples if URIRef(shex_triple[0]) == s], verbose)
    return candidate_classes

def validate_sparql_against_shex(variables, sparql_triples, shex_triples, verbose):
    """
    Validate each SPARQL variable against ShEx triples.
    """
    candidate_shex_save = {}

    for var in variables:
        candidate_classes = find_matching_classes(var, sparql_triples, shex_triples, verbose)
        candidate_shex_save[var] = candidate_classes

        if candidate_classes == []:
            print(f"Error: No matching class found for variable {var}. This might indicate an issue with the SPARQL query.")

    return candidate_shex_save

In [14]:
sparql_text = "# Endpoint: https://rdfportal.org/sib/sparql\n#           https://sparql.uniprot.org/sparql\n# Description: Where can I find information about the binding sites of the ACE2 gene?\n# Parameter: uniprot_gene_name: (example: ACE2)\n\nPREFIX core: <http://purl.uniprot.org/core/>\nPREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX faldo: <http://biohackathon.org/resource/faldo#>\nPREFIX skos: <http://www.w3.org/2004/02/skos/core#>\n\nSELECT DISTINCT ?binding_site_annotation_ligand ?binding_site_annotation_begin_position ?binding_site_annotation_end_position ?uniprot_gene_name\nWHERE {\n    VALUES ?uniprot_gene_name { \"ACE2\" }\n    VALUES ?UniProt__class { core:Protein chembl:UniprotRef }\n    ?UniProt a ?UniProt__class ;\n        core:annotation / core:ligand / rdfs:subClassOf ?binding_site_annotation_ligand ;\n        core:encodedBy / skos:prefLabel ?uniprot_gene_name .\n    ?BindingSiteAnnotation a core:Binding_Site_Annotation ;\n        core:range [\n            a faldo:Region ;\n            faldo:begin [\n                a faldo:Position ;\n                faldo:position ?binding_site_annotation_begin_position ;\n            ] ;\n            faldo:end [\n                a faldo:Position ;\n                faldo:position ?binding_site_annotation_end_position ;\n            ] ;\n        ] .\n}\nLIMIT 100\n"

In [15]:
print(sparql_text)

# Endpoint: https://rdfportal.org/sib/sparql
#           https://sparql.uniprot.org/sparql
# Description: Where can I find information about the binding sites of the ACE2 gene?
# Parameter: uniprot_gene_name: (example: ACE2)

PREFIX core: <http://purl.uniprot.org/core/>
PREFIX chembl: <http://rdf.ebi.ac.uk/terms/chembl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX faldo: <http://biohackathon.org/resource/faldo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?binding_site_annotation_ligand ?binding_site_annotation_begin_position ?binding_site_annotation_end_position ?uniprot_gene_name
WHERE {
    VALUES ?uniprot_gene_name { "ACE2" }
    VALUES ?UniProt__class { core:Protein chembl:UniprotRef }
    ?UniProt a ?UniProt__class ;
        core:annotation / core:ligand / rdfs:subClassOf ?binding_site_annotation_ligand ;
        core:encodedBy / skos:prefLabel ?uniprot_gene_name .
    ?BindingSiteAnnotation a core:Binding_Site_Annotation ;
        core:ra

In [20]:
q = prepareQuery(sparql_text)

variables, projected_vars, internal_vars = extract_variables_from_query(q)

In [21]:
triples = extract_triples(q.algebra)

In [22]:
results = validate_sparql_against_shex(variables, triples, shex_triples, False)
results

<FunctionAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<BindingSiteAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<CatalyticActivityAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<MutagenesisAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NaturalVariantAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NcbiGeneShape> does not look like a valid URI, trying to serialize this will break.
<ReactomeShape> does not look like a valid URI, trying to serialize this will break.
<PdbShape> does not look like a valid URI, trying to serialize this will break.
<GOShape> does not look like a valid URI, trying to serialize this will break.
<CitationShape> does not look like a valid URI, trying to serialize this will break.
<RheaShape> does not look like a valid URI, trying to serialize this will break.


{'UniProt': ['<UniProtShape>',
  '<UniProtShape>',
  '<FunctionAnnotationShape>',
  '<CatalyticActivityAnnotationShape>',
  '<MutagenesisAnnotationShape>',
  '<NaturalVariantAnnotationShape>',
  '<BindingSiteAnnotationShape>',
  '<NcbiGeneShape>',
  '<ReactomeShape>',
  '<PdbShape>',
  '<GOShape>',
  '<CitationShape>',
  '<CitationShape>',
  '<CitationShape>',
  '<CitationShape>',
  '<CitationShape>',
  '<CitationShape>',
  '<CitationShape>'],
 'BindingSiteAnnotation': ['<BindingSiteAnnotationShape>'],
 'binding_site_annotation_end_position': [-1],
 'uniprot_gene_name': [-1],
 'UniProt__class': ['http://purl.uniprot.org/core/Protein',
  'http://rdf.ebi.ac.uk/terms/chembl#UniprotRef',
  'http://purl.uniprot.org/core/Function_Annotation',
  'http://purl.uniprot.org/core/Catalytic_Activity_Annotation',
  'http://purl.uniprot.org/core/Mutagenesis_Annotation',
  'http://purl.uniprot.org/core/Natural_Variant_Annotation',
  'http://purl.uniprot.org/core/Binding_Site_Annotation',
  'http://pur

In [17]:
variables

{'BindingSiteAnnotation',
 'UniProt',
 'UniProt__class',
 'binding_site_annotation_begin_position',
 'binding_site_annotation_end_position',
 'binding_site_annotation_ligand',
 'uniprot_gene_name'}

In [18]:
triples

[(rdflib.term.BNode('N12ee6576b9aa4b09a38db8975324ca03'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://biohackathon.org/resource/faldo#Region')),
 (rdflib.term.BNode('N12ee6576b9aa4b09a38db8975324ca03'),
  rdflib.term.URIRef('http://biohackathon.org/resource/faldo#begin'),
  rdflib.term.BNode('Nc8bcd47ec3a44d0fbf81cc29877b3ee3')),
 (rdflib.term.BNode('N12ee6576b9aa4b09a38db8975324ca03'),
  rdflib.term.URIRef('http://biohackathon.org/resource/faldo#end'),
  rdflib.term.BNode('N52305ce3ad5c4f3bb036a3234b27a65d')),
 (rdflib.term.Variable('BindingSiteAnnotation'),
  rdflib.term.URIRef('http://purl.uniprot.org/core/range'),
  rdflib.term.BNode('N12ee6576b9aa4b09a38db8975324ca03')),
 (rdflib.term.BNode('N52305ce3ad5c4f3bb036a3234b27a65d'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://biohackathon.org/resource/faldo#Position')),
 (rdflib.term.BNode('Nc8bcd47ec3a44d0fbf81cc29877b3e

In [19]:
for sparql in sparqls:
    print(sparql["id"])
    # print(sparql["sparql"])
    q = prepareQuery(sparql["sparql"])
    # 抽出された変数を表示
    variables, projected_vars, internal_vars = extract_variables_from_query(q)
    
    # クエリのアルジェブラからトリプルを抽出
    triples = extract_triples(q.algebra)
    results = validate_sparql_against_shex(variables, triples, shex_triples, False)
    print(results)
    # if there is [] in results dict then false
    result_label = None
    if [] in list(results.values()):
        result_label = False
    else:
        result_label = True
    print("label compare", sparql["label"] == result_label, "\tCheck label", result_label, "\tAnswer label", result_label, )
    print("\n\n")

<FunctionAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<BindingSiteAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<CatalyticActivityAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<MutagenesisAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NaturalVariantAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NcbiGeneShape> does not look like a valid URI, trying to serialize this will break.
<ReactomeShape> does not look like a valid URI, trying to serialize this will break.
<PdbShape> does not look like a valid URI, trying to serialize this will break.
<GOShape> does not look like a valid URI, trying to serialize this will break.
<CitationShape> does not look like a valid URI, trying to serialize this will break.
<RheaShape> does not look like a valid URI, trying to serialize this will break.
<Function

1
{'uniprot_pdb': ['<PdbShape>'], 'UniProt__class': ['http://purl.uniprot.org/core/Protein', 'http://rdf.ebi.ac.uk/terms/chembl#UniprotRef', 'http://purl.uniprot.org/core/Function_Annotation', 'http://purl.uniprot.org/core/Catalytic_Activity_Annotation', 'http://purl.uniprot.org/core/Mutagenesis_Annotation', 'http://purl.uniprot.org/core/Natural_Variant_Annotation', 'http://purl.uniprot.org/core/Binding_Site_Annotation', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Structure_Resource', 'http://www.w3.org/2002/07/owl#Class', 'http://purl.uniprot.org/core/Journal_Citation', 'http://purl.uniprot.org/core/Submission_Citation', 'http://purl.uniprot.org/core/Thesis_Citation', 'http://purl.uniprot.org/core/Observation_Citation', 'http://purl.uniprot.org/core/Patent_Citation', 'http://purl.uniprot.org/core/Book_Citation', 'http://purl.uniprot.org/core/Electronic_Citation'], 'UniProt': ['<NaturalVariantAnnotationShape>', '<UniPr

<FunctionAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<BindingSiteAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<CatalyticActivityAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<MutagenesisAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NaturalVariantAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NcbiGeneShape> does not look like a valid URI, trying to serialize this will break.
<ReactomeShape> does not look like a valid URI, trying to serialize this will break.
<PdbShape> does not look like a valid URI, trying to serialize this will break.
<GOShape> does not look like a valid URI, trying to serialize this will break.
<CitationShape> does not look like a valid URI, trying to serialize this will break.
<RheaShape> does not look like a valid URI, trying to serialize this will break.
<Function

Error: No matching class found for variable UniProt. This might indicate an issue with the SPARQL query.
{'uniprot_pdb': ['<PdbShape>'], 'y': [-1], 'x': [-1], 'UniProt__class': ['http://purl.uniprot.org/core/Protein', 'http://rdf.ebi.ac.uk/terms/chembl#UniprotRef', 'http://purl.uniprot.org/core/Function_Annotation', 'http://purl.uniprot.org/core/Catalytic_Activity_Annotation', 'http://purl.uniprot.org/core/Mutagenesis_Annotation', 'http://purl.uniprot.org/core/Natural_Variant_Annotation', 'http://purl.uniprot.org/core/Binding_Site_Annotation', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Structure_Resource', 'http://www.w3.org/2002/07/owl#Class', 'http://purl.uniprot.org/core/Journal_Citation', 'http://purl.uniprot.org/core/Submission_Citation', 'http://purl.uniprot.org/core/Thesis_Citation', 'http://purl.uniprot.org/core/Observation_Citation', 'http://purl.uniprot.org/core/Patent_Citation', 'http://purl.uniprot.org/cor

<CatalyticActivityAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<MutagenesisAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NaturalVariantAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<NcbiGeneShape> does not look like a valid URI, trying to serialize this will break.
<ReactomeShape> does not look like a valid URI, trying to serialize this will break.
<PdbShape> does not look like a valid URI, trying to serialize this will break.
<GOShape> does not look like a valid URI, trying to serialize this will break.
<CitationShape> does not look like a valid URI, trying to serialize this will break.
<RheaShape> does not look like a valid URI, trying to serialize this will break.
<FunctionAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<BindingSiteAnnotationShape> does not look like a valid URI, trying to serialize this will break.
<Catalyti

{'FunctionAnnotation': ['<FunctionAnnotationShape>'], 'uniprot_gene_name': [-1], 'UniProt__class': ['http://purl.uniprot.org/core/Protein', 'http://rdf.ebi.ac.uk/terms/chembl#UniprotRef', 'http://purl.uniprot.org/core/Function_Annotation', 'http://purl.uniprot.org/core/Catalytic_Activity_Annotation', 'http://purl.uniprot.org/core/Mutagenesis_Annotation', 'http://purl.uniprot.org/core/Natural_Variant_Annotation', 'http://purl.uniprot.org/core/Binding_Site_Annotation', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Resource', 'http://purl.uniprot.org/core/Structure_Resource', 'http://www.w3.org/2002/07/owl#Class', 'http://purl.uniprot.org/core/Journal_Citation', 'http://purl.uniprot.org/core/Submission_Citation', 'http://purl.uniprot.org/core/Thesis_Citation', 'http://purl.uniprot.org/core/Observation_Citation', 'http://purl.uniprot.org/core/Patent_Citation', 'http://purl.uniprot.org/core/Book_Citation', 'http://purl.uniprot.org/core/Electronic_Citation'], 'UniPro